In [1]:
# default_exp core

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/MyDrive/timeseries_feature_engineering'
!pip3 install -e . -q
!pip3 install nbdev -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/timeseries_feature_engineering


# module name here

> API details.

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export 
from __future__ import print_function, division

import random


class Card:
    """Represents a standard playing card.
    
    Attributes:
      suit: integer 0-3
      rank: integer 1-13
    """

    suit_names = ["Clubs", "Diamonds", "Hearts", "Spades"]
    rank_names = [None, "Ace", "2", "3", "4", "5", "6", "7", 
              "8", "9", "10", "Jack", "Queen", "King"]

    def __init__(self, suit=0, rank=2):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        """Returns a human-readable string representation."""
        return '%s of %s' % (Card.rank_names[self.rank],
                             Card.suit_names[self.suit])

    def __eq__(self, other) -> bool:
        """Checks whether self and other have the same rank and suit.
        """
        return self.suit == other.suit and self.rank == other.rank

    def __lt__(self, other) -> bool:
        """Compares this card to other, first by suit, then rank.
        """
        t1 = self.suit, self.rank
        t2 = other.suit, other.rank
        return t1 < t2
    
    def __repr__(self): return self.__str__()
    
    def foo(): pass

Card is a class that represents a single card in a deck of cards. For example:

In [5]:
Card(suit=2, rank=11)

Jack of Hearts

In [6]:
c = Card(suit=1, rank=3)
assert str(c) == '3 of Diamonds'

c2 = Card(suit=2, rank=11)
assert str(c2) == 'Jack of Hearts'

In [7]:
assert c2 > c

In [9]:
show_doc(Card.__eq__)

<h4 id="Card.__eq__" class="doc_header"><code>Card.__eq__</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>Card.__eq__</code>(**`other`**)

Checks whether self and other have the same rank and suit.
        

In [10]:
card1 = Card(suit=1, rank=3)
card2 = Card(suit=1, rank=3)
assert card1 == card2

In [20]:
!git push origin master

Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [18]:
!git remote set-url origin git@github.com:shanaka-desoysa/timeseries_feature_engineering.git

In [21]:
!git fetch

Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
